In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['font.family'] = 'Gulim'

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
from workalendar.asia import SouthKorea
import pendulum

In [3]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

## 전처리
- 일자에서 월과 일을 분리
- 요일을 레이블 인코딩화(EDA로 요일의 중요도 순 파악)
- 월 별, 일 별 중식 석식 수요 차이 파악

In [4]:
train['월'] = pd.DatetimeIndex(train['일자']).month
test['월'] = pd.DatetimeIndex(test['일자']).month
train['주'] = pd.DatetimeIndex(train['일자']).week
test['주'] = pd.DatetimeIndex(test['일자']).week
train['일'] = pd.DatetimeIndex(train['일자']).day
test['일'] = pd.DatetimeIndex(test['일자']).day

train['출근'] = train['본사정원수']-(train['본사휴가자수']+train['본사출장자수']+train['현본사소속재택근무자수'])
train['휴가비율'] = train['본사휴가자수']/train['본사정원수']
train['출장비율'] = train['본사출장자수']/train['본사정원수']
train['야근비율'] = train['본사시간외근무명령서승인건수']/train['출근']
train['재택비율'] = train['현본사소속재택근무자수']/train['본사정원수']

test['출근'] = test['본사정원수']-(test['본사휴가자수']+test['본사출장자수']+test['현본사소속재택근무자수'])
test['휴가비율'] = test['본사휴가자수']/test['본사정원수']
test['출장비율'] = test['본사출장자수']/test['본사정원수']
test['야근비율'] = test['본사시간외근무명령서승인건수']/test['출근']
test['재택비율'] = test['현본사소속재택근무자수']/test['본사정원수']

train['식사가능자수'] = train['본사정원수'] - train['본사휴가자수'] - train['현본사소속재택근무자수']
test['식사가능자수'] = test['본사정원수'] - test['본사휴가자수'] - test['현본사소속재택근무자수']

# train['중식참여율'] = train['중식계'] / train['식사가능자수']

<ipython-input-4-b11c832c5082>:3: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  train['주'] = pd.DatetimeIndex(train['일자']).week
<ipython-input-4-b11c832c5082>:4: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  test['주'] = pd.DatetimeIndex(test['일자']).week


In [5]:
month_rank4dinner = {
    1: 11,
    2: 2,
    3: 1,
    4: 4,
    5: 7,
    6: 6,
    7: 10,
    8: 8,
    9: 5,
    10: 3,
    11: 9,
    12: 12
}
train['월(석식)'] = train['월'].map(month_rank4dinner)
test['월(석식)'] = test['월'].map(month_rank4dinner)

month_rank4lunch = {
    1: 3,
    2: 1,
    3: 2,
    4: 6,
    5: 7,
    6: 8,
    7: 10,
    8: 9,
    9: 5,
    10: 4,
    11: 11,
    12: 12
}
train['월(중식)'] = train['월'].map(month_rank4lunch)
test['월(중식)'] = test['월'].map(month_rank4lunch)

weekday_rank4dinner = {
    '월': 1,
    '화': 2,
    '수': 4,
    '목': 3,
    '금': 5,
}

weekday_rank4lunch = {
    '월': 1,
    '화': 2,
    '수': 3,
    '목': 4,
    '금': 5,
}

train['요일(석식)'] = train['요일'].map(weekday_rank4dinner)
test['요일(석식)'] = test['요일'].map(weekday_rank4dinner)

train['요일(중식)'] = train['요일'].map(weekday_rank4lunch)
test['요일(중식)'] = test['요일'].map(weekday_rank4lunch)

In [6]:
rank = pd.DataFrame(range(1,53))
week_rank_lunch = pd.pivot_table(train,values='중식계',index='주').sort_values(by='중식계').reset_index().drop('중식계',axis=1)
week_rank_dinner = pd.pivot_table(train,values='석식계',index='주').sort_values(by='석식계').reset_index().drop('석식계',axis=1)


week_rank4lunch = {}
for i in range(len(rank)):
    week_rank4lunch[week_rank_lunch['주'][i]] = rank[0][i]


week_rank4dinner = {}
for i in range(len(rank)):
    week_rank4dinner[week_rank_dinner['주'][i]] = rank[0][i]
    
    
train['주(중식)'] = train['주'].map(week_rank4lunch)
test['주(중식)'] = test['주'].map(week_rank4lunch)

train['주(석식)'] = train['주'].map(week_rank4dinner)
test['주(석식)'] = test['주'].map(week_rank4dinner)

In [7]:
def is_holiday(date):
    holidays = list(map(str, pd.Series(np.array(SouthKorea().holidays(int(date[:4])))[:, 0])))
    
    yesterday = str(np.datetime64(date) - 1)
    tomorrow = str(np.datetime64(date) + 1)

    if tomorrow in holidays and yesterday in holidays:
        return 3
    if tomorrow in holidays:
        return 2
    elif yesterday in holidays:
        return 1
    else : 
        return 0

def week_of_month(x):
    dt = pendulum.parse(x)
    
    wom = dt.week_of_month
    if wom < 0:
        wom += 52
    return wom
    

df = pd.concat([train[['본사정원수', '일자']], test[['본사정원수', '일자']]])
df['년월'] = df['일자'].apply(lambda x : x[:7])
df = df[['년월', '본사정원수']].groupby(by=['년월'], as_index=False).mean()

def member_change(date):
    this_month = date[:7]
    last_month = str(np.datetime64(this_month) - 1)
    
    this_month_member = int(df[df['년월'] == this_month]['본사정원수'])
    last_month_member = int(df[df['년월'] == last_month]['본사정원수'])
    
    
    return  this_month_member - last_month_member

train['공휴일전후'] = train['일자'].apply(is_holiday)
test['공휴일전후'] = test['일자'].apply(is_holiday)

train['몇주차'] = train['일자'].apply(week_of_month)
test['몇주차'] = test['일자'].apply(week_of_month)

train = train[train['일자'] > '2016-03']
train['인원변화'] = train['일자'].apply(member_change)
test['인원변화'] = test['일자'].apply(member_change)

## 공휴일 변수 생성

In [8]:
train.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '조식메뉴', '중식메뉴', '석식메뉴', '중식계', '석식계', '월', '주', '일',
       '출근', '휴가비율', '출장비율', '야근비율', '재택비율', '식사가능자수', '월(석식)', '월(중식)',
       '요일(석식)', '요일(중식)', '주(중식)', '주(석식)', '공휴일전후', '몇주차', '인원변화'],
      dtype='object')

In [9]:
# 메뉴 변수 없이 사용할떄 해당 코드 사용['공휴일전후', '몇주차', '인원변화']

lunch_train = train[['공휴일전후', '몇주차', '인원변화', '요일(중식)','월(중식)','일','주(중식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수','본사시간외근무명령서승인건수']]
lunch_test = test[['공휴일전후', '몇주차', '인원변화', '요일(중식)','월(중식)','일','주(중식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수','본사시간외근무명령서승인건수']]

dinner_train= train[['공휴일전후', '몇주차', '인원변화', '요일(석식)','월(석식)','일','주(석식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수','본사시간외근무명령서승인건수']]
dinner_test = test[['공휴일전후', '몇주차', '인원변화', '요일(석식)','월(석식)','일','주(석식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수','본사시간외근무명령서승인건수']]

In [10]:
lunch_train.columns

Index(['공휴일전후', '몇주차', '인원변화', '요일(중식)', '월(중식)', '일', '주(중식)', '출근', '휴가비율',
       '출장비율', '야근비율', '재택비율', '본사출장자수', '본사휴가자수', '식사가능자수', '본사시간외근무명령서승인건수'],
      dtype='object')

In [11]:
print(lunch_train.shape)
print(lunch_test.shape)

(1187, 16)
(50, 16)


In [12]:
print(dinner_train.shape)
print(dinner_test.shape)

(1187, 16)
(50, 16)


In [13]:
cat_features = [f for f in lunch_train.columns if lunch_train[f].dtype == 'object']

def column_index(df, cat_features):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols, cat_features, sorter=sidx)]

cat_features_idx = column_index(lunch_train, cat_features)    
print("Cat features are: %s" % [f for f in cat_features])
print(cat_features_idx)

Cat features are: []
[]


In [14]:
y_lunch = train[['중식계']]
y_dinner = train[['석식계']]

#### 분포 확인 및 분포 조정

# 중식 예측모델

In [15]:
from xgboost import XGBRegressor

In [16]:
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import mean_absolute_error as MAE

In [17]:
X_train, X_valid, y_train, y_valid = train_test_split(lunch_train, y_lunch, test_size=0.2)

In [18]:
reg_candidate = [1e-5, 1e-4, 1e-3, 1e-2, 0.1, 1, 5, 10, 100]

# space 정의, Hyperparameter의 이름을 key 값으로 입력
space={'max_depth': hp.quniform("max_depth", 5, 15, 1),
       'learning_rate': hp.quniform ('learning_rate', 0.01, 0.05, 0.005),
       'reg_alpha' : hp.choice('reg_alpha', reg_candidate),
       'reg_lambda' : hp.choice('reg_lambda', reg_candidate),
       'subsample': hp.quniform('subsample', 0.6, 1, 0.05),
       'colsample_bytree' : hp.quniform('colsample_bytree', 0.6, 1, 0.05),
       'min_child_weight' : hp.quniform('min_child_weight', 1, 10, 1),
       'n_estimators': hp.quniform('n_estimators', 200, 1500, 100)
      }
y_lunch = np.array(y_lunch).reshape(-1,1)

def hyperparameter_tuning_L(space):
    n_fold = 4
    kf = KFold(n_splits=n_fold, random_state=42, shuffle=True)
    mae_log = 0
    
    for train_index, test_index in kf.split(lunch_train):
        X_train, X_valid = lunch_train.iloc[train_index], lunch_train.iloc[test_index]
        y_train, y_valid = y_lunch[train_index], y_lunch[test_index]

        model=XGBRegressor(n_estimators =int(space['n_estimators']), 
                           max_depth = int(space['max_depth']), 
                           learning_rate = space['learning_rate'],
                           reg_alpha = space['reg_alpha'],
                           reg_lambda = space['reg_lambda'],
                           subsample = space['subsample'],
                           colsample_bytree = space['colsample_bytree'], 
                           min_child_weight = int(space['min_child_weight']),
                           random_state=42, 
                          )

        evaluation = [(X_train, y_train), (X_valid, y_valid)]

        model.fit(X_train, y_train,
                  eval_set=evaluation, 
                  eval_metric="mae",
                  early_stopping_rounds=20,
                  verbose=0)

        pred = model.predict(X_valid)
        mae= MAE(y_valid, pred)   
        
        mae_log += mae/n_fold
    # 평가 방식 선정
    return {'loss':mae_log, 'status': STATUS_OK, 'model': model}

In [39]:
# Trials 객체 선언합니다.
trials = Trials()
# best에 최적의 하이퍼 파라미터를 return 받습니다.
best_L = fmin(fn=hyperparameter_tuning_L,
              space=space,
              algo=tpe.suggest,
              max_evals=200, # 최대 반복 횟수를 지정합니다.
              trials=trials,
              rstate= np.random.seed(42))

# 최적화된 결과를 int로 변환해야하는 파라미터는 타입 변환을 수행합니다.
best_L['max_depth'] = int(best_L['max_depth'])
best_L['min_child_weight'] = int(best_L['min_child_weight'])
best_L['n_estimators'] = int(best_L['n_estimators'])
best_L['reg_alpha'] = reg_candidate[int(best_L['reg_alpha'])]
best_L['reg_lambda'] = reg_candidate[int(best_L['reg_lambda'])]
best_L['random_state'] = 42
print(best_L)

100%|███████████████████████████████████████████████| 200/200 [06:37<00:00,  1.99s/trial, best loss: 65.15730193531516]
{'colsample_bytree': 1.0, 'learning_rate': 0.045, 'max_depth': 7, 'min_child_weight': 8, 'n_estimators': 800, 'reg_alpha': 0.01, 'reg_lambda': 1, 'subsample': 0.7000000000000001, 'random_state': 42}


In [40]:
lunch_model = XGBRegressor(**best_L)
lunch_model.fit(lunch_train, y_lunch)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1.0, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.045, max_delta_step=0, max_depth=7,
             min_child_weight=8, missing=nan, monotone_constraints='()',
             n_estimators=800, n_jobs=8, num_parallel_tree=1, random_state=42,
             reg_alpha=0.01, reg_lambda=1, scale_pos_weight=1,
             subsample=0.7000000000000001, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [41]:
lunch_predict = lunch_model.predict(lunch_test)
lunch_predict

array([ 987.0721 ,  925.5664 ,  574.9289 , 1287.2858 , 1058.0447 ,
       1064.3463 ,  947.3401 ,  678.55347, 1308.8759 , 1102.2638 ,
        847.9991 , 1306.2465 , 1115.3147 , 1081.7671 ,  937.07214,
        668.40485, 1226.1538 , 1100.4045 ,  938.4504 ,  893.7255 ,
        659.5558 , 1203.507  , 1027.8278 ,  943.44086,  645.87335,
       1290.6382 , 1150.6448 , 1068.634  ,  971.3472 ,  696.14185,
       1281.4982 , 1023.01404, 1045.217  ,  933.1107 ,  634.32227,
       1217.2712 ,  991.4329 ,  936.5295 ,  879.3581 ,  591.96576,
       1187.7836 ,  988.056  ,  975.9513 ,  864.7479 ,  652.5291 ,
       1225.3322 , 1074.6904 ,  978.58844,  896.70935,  633.7066 ],
      dtype=float32)

In [47]:
y_dinner = np.array(y_dinner).reshape(-1,1)

def hyperparameter_tuning_D(space):
    n_fold = 4
    kf = KFold(n_splits=n_fold, random_state=42, shuffle=True)
    mae_log = 0
    
    for train_index, test_index in kf.split(dinner_train):
        X_train, X_valid = dinner_train.iloc[train_index], dinner_train.iloc[test_index]
        y_train, y_valid = y_dinner[train_index], y_dinner[test_index]

        model=XGBRegressor(n_estimators =int(space['n_estimators']), 
                           max_depth = int(space['max_depth']), 
                           learning_rate = 0.05,
                           reg_alpha = space['reg_alpha'],
                           reg_lambda = space['reg_lambda'],
                           subsample = space['subsample'],
                           colsample_bytree = space['colsample_bytree'], 
                           min_child_weight = int(space['min_child_weight']),
                           random_state=42, 
                          )

        evaluation = [(X_train, y_train), (X_valid, y_valid)]

        model.fit(X_train, y_train,
                  eval_set=evaluation, 
                  eval_metric="mae",
                  early_stopping_rounds=20,
                  verbose=0)

        pred = model.predict(X_valid)
        mae= MAE(y_valid, pred)   
        
        mae_log += mae/n_fold

    return {'loss':mae_log, 'status': STATUS_OK, 'model': model}

In [48]:
trials = Trials()

best_D = fmin(fn=hyperparameter_tuning_D,
              space=space,
              algo=tpe.suggest,
              max_evals=200, # 최대 반복 횟수를 지정합니다.
              trials=trials,
              rstate= np.random.seed(42))
              
best_D['max_depth'] = int(best_D['max_depth'])
best_D['min_child_weight'] = int(best_D['min_child_weight'])
best_D['n_estimators'] = int(best_D['n_estimators'])
best_D['reg_alpha'] = reg_candidate[int(best_D['reg_alpha'])]
best_D['reg_lambda'] = reg_candidate[int(best_D['reg_lambda'])]
best_D['random_state'] = 42
print (best_D)

 42%|███████████████████▉                            | 83/200 [02:22<03:20,  1.71s/trial, best loss: 54.27227861434967]


KeyboardInterrupt: 

In [44]:
dinner_model = XGBRegressor(**best_D)
dinner_model.fit(dinner_train, y_dinner)

dinner_predict = dinner_model.predict(dinner_test)

In [58]:
submission = pd.read_csv('../data/sample_submission.csv')

submission.iloc[:,1] = lunch_predict
submission.iloc[:,2] = dinner_predict
submission.head()

,일자,중식계,석식계
0,2021-01-27,987.072083,280.855011
1,2021-01-28,925.566406,399.208191
2,2021-01-29,574.928894,217.562180
3,2021-02-01,1287.285767,540.310852
4,2021-02-02,1058.044678,480.341980


In [63]:
submission.mean()

중식계    974.384827
석식계    475.232971
dtype: float32

In [60]:
def compare_ans(DIR):
    answer = pd.read_csv(DIR)
    lunch_answer = np.array(answer.iloc[:,1])

    dinner_answer = np.array(answer.iloc[:,2])
    
    lunch_MAE = abs(submission.iloc[:,1] - lunch_answer).mean()
    dinner_MAE = abs(submission.iloc[:,2] - dinner_answer).mean()
    
    print("lunch_MAE : ", lunch_MAE)
    print("dinner_MAE : ", dinner_MAE)
    print("total_MAE : ", (lunch_MAE+dinner_MAE)/2)

In [61]:
import glob

files = glob.glob('../submission/*')

for file in files[:-1]:
    print(file, compare_ans(file), end="\n\n" )

lunch_MAE :  27.249404003531243
dinner_MAE :  36.29071565157031
total_MAE :  31.770059827550774
../submission\20210624_pycaret_(4-2)_제출해봐야할거.csv None

lunch_MAE :  52.288494465300474
dinner_MAE :  43.794179843398524
total_MAE :  48.0413371543495
../submission\20210625_cat.csv None

lunch_MAE :  48.20858501519929
dinner_MAE :  46.05437324579555
total_MAE :  47.131479130497425
../submission\20210625_cat_10fold.csv None

lunch_MAE :  48.5874154198538
dinner_MAE :  46.28641963324848
total_MAE :  47.43691752655114
../submission\20210627_cat_15_KJH.csv None

lunch_MAE :  64.41096871433142
dinner_MAE :  61.84023420729628
total_MAE :  63.12560146081385
../submission\20210627_cat_after_2020.csv None

lunch_MAE :  48.5874154198538
dinner_MAE :  46.28641963324848
total_MAE :  47.43691752655114
../submission\20210627_cat_KJH.csv None

lunch_MAE :  48.5874154198538
dinner_MAE :  46.28641963324848
total_MAE :  47.43691752655114
../submission\20210627_pycaret_KJH.csv None

lunch_MAE :  34.96786352176

# 저장

In [62]:
import datetime
today = str(datetime.datetime.now().date()).replace("-","")
print("오늘 날짜 : " + today)

submission.to_csv(f'../submission/{today}_hyperOpt_xgb2.csv', index =False)

오늘 날짜 : 20210722


In [67]:
import math
    
math.cos(80/180*math.pi)*50

8.68240888334652

In [69]:
1/3 + 2/3

0.3339388032056354